In [1]:
import rospy
import actionlib
import actionlib.msg
import assignment_2_2023.msg
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import tf

from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningResult, PlanningFeedback, PlanningActionFeedback
from std_srvs.srv import *
from nav_msgs.msg import Odometry
from actionlib import GoalStatusArray, GoalStatus, GoalID
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2023.msg import Pos_Vel
from ipywidgets import Button, Layout
from IPython.display import display
from ipywidgets import widgets, Button, IntSlider, HTML, VBox
from tf.transformations import quaternion_matrix
from matplotlib.animation import FuncAnimation

%matplotlib widget

In [2]:
"""class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.xdata, self.ydata = [] , []

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln,
        
    def odom_callback(self, msg):
        self.ydata.append(msg.pose.pose.position.y)
        self.xdata.append(msg.pose.pose.position.x)
        
    def update_plot(self, frame):
        self.ln.set_data(self.xdata, self.ydata)
        return self.ln,
"""

"class Visualiser:\n    def __init__(self):\n        self.fig, self.ax = plt.subplots()\n        self.ln, = plt.plot([], [], 'ro')\n        self.xdata, self.ydata = [] , []\n\n    def plot_init(self):\n        self.ax.set_xlim(-10, 10)\n        self.ax.set_ylim(-10, 10)\n        return self.ln,\n        \n    def odom_callback(self, msg):\n        self.ydata.append(msg.pose.pose.position.y)\n        self.xdata.append(msg.pose.pose.position.x)\n        \n    def update_plot(self, frame):\n        self.ln.set_data(self.xdata, self.ydata)\n        return self.ln,\n"

In [3]:
class Visualiser:
    def __init__(self):
        out = widgets.Output()
        plt.ioff()
        self.fig, (self.ax1, self.ax2) = plt.subplots(1,2,figsize=(7,3),constrained_layout=True)
        self.fig.canvas.toolbar_position = 'bottom'
        plt.ion()
        with out:
            display(self.fig.canvas)
        # Bar graph and button class
        self.bar_data = [0,0]
        self.bar_labels = ['Cancel','Reached']
        self.bars = self.ax2.bar(self.bar_labels, self.bar_data)
        # Title of the subplot
        self.ax1.set_title('Robot Position')
        self.ax2.set_title('Bar Graph')
        plt.ioff()
        self.ln, = self.ax1.plot([], [], 'ro')
        self.xdata, self.ydata = [] , []
        plt.ion()
        
        #display(out)
        
    def plot_init(self):
        self.ax1.set_xlim(-10, 10)
        self.ax1.set_ylim(-10, 10)
        self.ax2.set_xlim()
        self.ax2.set_ylim(0,5)
        return self.ln, self.bars
    def odom_callback(self, msg):
        self.ydata.append(msg.pose.pose.position.y)
        self.xdata.append(msg.pose.pose.position.x)
        cancelled = len(del_x_target)
        reached = len(x_reached)
        y = [cancelled,reached]
        self.update_bar(y)
 
    def update_plot(self, frame):
        self.ln.set_data(self.xdata, self.ydata)
        self.bars[0].set_height(self.bar_data[0])
        self.bars[1].set_height(self.bar_data[1])
        return self.ln, self.bars
    def update_bar(self, y):
        self.bar_data = y

In [4]:
# Global variables

global publisher
publisher = None
global client
client = None

global x_data
x_data = []
y_data = []

x_reached = []
y_reached = []

del_x_target = []
del_y_target = []


def client_request(button, x_in, y_in):

    global client
    global x
    global y
    # Create a SimpleActionClient
    client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
    client.wait_for_server()
    
    
    x = x_in.value
            
    y = y_in.value
    
    x_data.append(x)
    y_data.append(y)
    
    target = assignment_2_2023.msg.PlanningGoal()
            
    target.target_pose.pose.position.x = x
      
    target.target_pose.pose.position.y = y
            
    # Send the new goal
    client.send_goal(target)   
        
def status_callback(msg):

    position = msg.feedback.stat
    
    if position == "Target reached!":
        print("reached!") 
        x_reached.append(x)
        y_reached.append(y)
        
        
def send_goal(button, x_input, y_input):
    global publisher  # Use the global publisher variable
    client_request(button, x_input, y_input)
    #publisher.publish(...)  # Publish the message here

def cancel_goal(button):
    
    global client
    global x_data
    if (len(x_data) == 0):
        print("No target to delete")
    else:
        x = rospy.get_param('/des_pos_x')
        y = rospy.get_param('/des_pos_y')
        if ( x != x_reached[-1]  and y != y_reached[-1]):
            client.cancel_goal()
            del_x_target.append(x)
            del_y_target.append(y)
            print("Goal has been cancelled!")
        
    
    
    
    
    
    

def callback(msg):
    
    global publisher
    
    # Extract position, linear velocity and angular velocity from the message
    pos = msg.pose.pose.position
    vel = msg.twist.twist.linear
    ang = msg.twist.twist.angular
    
    # Create a new Pos_Vel message
    posvel = Pos_Vel()
    
    # Give the value related to position and velocity to the message
    '''
    Uncomment the following lines if you want the position of the robot in feet
    '''
    #posvel.x = pos.x*3.28
    #posvel.y = pos.y*3.28
    
    '''
    Position in meters
    Velocity in m/s
    '''
    posvel.x = pos.x
    posvel.y = pos.y
    #posvel.z = pos.z    Uncomment this row if you want also the z coordinate and add in the Pos_Vel.msg what is written in the brackets (float64 z)
    posvel.vel_x = vel.x
    posvel.vel_z = ang.z
    
    
    # Publish the message
    publisher.publish(posvel)
    
"""    
def odom_callback(msg):
    x_data.append(msg.pose.pose.position.x)
    y_data.append(msg.pose.pose.position.y)
"""

def graph(button):
    np_x_plot = np.array(x_data)
    np_y_plot = np.array(y_data)
    
    fig_trajectory = plt.plot(np_x_plot, np_y_plot)
    plt.xlabel("x axis")
    plt.ylabel("y axis")
    plt.title("Robot position")

    plt.show()
    
def last_t_callback(button):        
    if (len(x_reached) == 0):
        print("No target available")
    else:
        print("Last Targets: ")
        for i in range(len(x_reached)):
            print("x: ", str(x_reached[i]), "y: ", str(y_reached[i]))
    
def deleted_goal(button):
    if (len(del_x_target) == 0):
        print("No target deleted yet")
    else:
        print("Deleted target: ", str(del_x_target[-1]), str(del_y_target[-1]))


In [5]:
out = widgets.Output()
with out:
    vis = Visualiser()
    sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
    ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit = True)
    plt.show(block=True)

In [6]:
def main():
    
    global publisher, sub, sub2
    # Initialize the node
    rospy.init_node('nodeF')
    
    x_input = widgets.IntSlider(value=0, min=-9, max=9, description="x ")
    y_input = widgets.IntSlider(value=0, min=-9, max=9, description="y ")    
    display(x_input)
    display(y_input)
    
    # Create a publisher
    publisher = rospy.Publisher("/pos_vel", Pos_Vel, queue_size=1)
    
    # Subscribe to the /odom topic
    rospy.Subscriber("/odom", Odometry, callback)
    
    rospy.Subscriber("/reaching_goal/feedback",PlanningActionFeedback,status_callback)
    
    send = widgets.Button(value=False, 
                          description="Send Goal", 
                          disabled=False, 
                          button_style='success',
                          icon='play')
    send.on_click(lambda b: send_goal(b, x_input, y_input))
        
    delete_button = widgets.Button(value=False, 
                                   description="Cancel Goal",
                                   disabled=False,
                                   button_style='danger',
                                   icon='times')
    
    delete_button.on_click(lambda b: cancel_goal(b))
    
    
    display(send)
    display(delete_button)
    
    """
    Button to see the graph related to the position of the robot
    """
    button = widgets.Button(description = "Show graph", 
                            disabled=False,
                            button_style='info')
    output = widgets.Output()
    display(button, output)
    
    button.on_click(graph)
    
    """
    Button to see the reached target
    """
    last_t = widgets.Button(description = "Show last target", 
                            disabled=False,
                            button_style='info')
    last_t_output = widgets.Output()
    
    display(last_t,last_t_output)
    
    last_t.on_click(last_t_callback)
    
    
    """
    Button to see the deleted target
    """
    deleted_target = widgets.Button(description = "Show deleted target",
                                    disabled=False,
                                    button_style='info')
    
    deleted_target_output = widgets.Output()
    display(deleted_target, deleted_target_output)
    
    deleted_target.on_click(deleted_goal)
    
    display(out)
    
# Entry point
if __name__ == '__main__':
    main()


IntSlider(value=0, description='x ', max=9, min=-9)

IntSlider(value=0, description='y ', max=9, min=-9)

Button(button_style='success', description='Send Goal', icon='play', style=ButtonStyle())

Button(button_style='danger', description='Cancel Goal', icon='times', style=ButtonStyle())

Button(button_style='info', description='Show graph', style=ButtonStyle())

...

Output()

Button(button_style='info', description='Show deleted target', style=ButtonStyle())

Output()

Output()

No target to delete
No target deleted yet
Last Targets: 
x:  2 y:  2
Goal has been cancelled!


In [7]:
  print(x_data)

[]
